In [1]:
def big_data(X,y,reqlen):
    Big_X = np.array(X)
    Big_Y = np.array(y)
    while len(Big_X) < reqlen:
        Big_X = np.concatenate((Big_X, Big_X))
        Big_Y = np.concatenate((Big_Y, Big_Y))
    print(Big_X.shape)
    indices = np.random.choice(Big_X.shape[0], reqlen, replace=False)
    new_X = Big_X[indices]
    new_Y = Big_Y[indices]
    return new_X, new_Y

# 1000 samples

In [2]:
from sklearn.datasets import load_boston
import sklearn.ensemble
import numpy as np
from sklearn.model_selection import train_test_split
from mosaicml import *
from mosaicml.constants import MLModelFlavours

import pandas as pd
from pandas import read_csv, DataFrame
from sklearn.preprocessing import Imputer
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from mosaicml.api import graph

# Importing the dataset
boston = load_boston()

# creating the model object
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=10)

X = boston.data
Y = boston.target

Big_X, Big_Y = big_data(X, Y,1000)

print(Big_X.shape)
print(Big_Y.shape)

train, test, labels_train, labels_test = train_test_split(Big_X, Big_Y, test_size=0.2, random_state=0)

rf.fit(train, labels_train)


# @scoring_func
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()

# to verify if score func works
import requests
req = requests.Request()
req.json = {"payload":train[10].tolist()}

score(rf,req)

from datetime import datetime
st = datetime.utcnow()
print(st,"\n")

y_pred=rf.predict(test)

register_model(rf,
                score,
                "boston_regression_1k",
                "boston_regression_1k",
                MLModelFlavours.sklearn,
                init_script="mkdir abc \\n cd abc \\n touch abc.txt",
                input_type="json",
                explain_ai=True,
               x_train=train,
               y_train=labels_train, 
               y_pred=y_pred,
               y_true=labels_test,
               features=list(boston.feature_names),
               original_features=list(boston.feature_names),
               model_type="regression",
               feature_names=list(boston.feature_names), 
               feature_ids=list(boston.feature_names), 
               target_names=['Housing_Price']
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


2021-02-22 12:19:45,287 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False
2021-02-22 12:19:45,290 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


[30/30] grid cells ████████████████████ Time elapsed: 1 seconds

2021-02-22 12:19:47,270 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False
2021-02-22 12:19:47,274 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


[25/25] grid cells ████████████████████ Time elapsed: 1 seconds

...

[30/30] grid cells ████████████████████ Time elapsed: 1 seconds

2021-02-22 12:20:07,698 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progress_bar=False
2021-02-22 12:20:07,701 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


[13/13] features ████████████████████ Time elapsed: 0 seconds

2021-02-22 12:20:09,070 - skater.core.global_interpretation.tree_surrogate - INFO - post pruning applied ...
2021-02-22 12:20:09,084 - skater.core.global_interpretation.tree_surrogate - INFO - Scorer used MAE
2021-02-22 12:20:09,085 - skater.core.global_interpretation.tree_surrogate - INFO - original score using base model 0.6611897110958216
2021-02-22 12:20:09,139 - skater.core.global_interpretation.tree_surrogate - INFO - Summary: childrens of the following nodes are removed [21]
2021-02-22 12:20:09,141 - skater.core.global_interpretation.tree_surrogate - INFO - Done generating prediction using the surrogate, shape (800,)
2021-02-22 12:20:09,143 - skater.core.global_interpretation.tree_surrogate - INFO - Done scoring, surrogate score 0.67; oracle score 0.469


{'created_by': 'mosaic.ai',
 'created_on': '2021-02-22T12:19:39+00:00',
 'deploymentstatus': False,
 'description': 'boston_regression_1k',
 'flavour': 'sklearn',
 'id': 'f09c5718-9f30-4a97-bc4d-1d61d6f24e87',
 'last_modified_by': 'mosaic.ai',
 'last_modified_on': '2021-02-22T12:19:39+00:00',
 'model_display': True,
 'name': 'boston_regression_1k',
 'project_id': 'f2090c34-4280-40b0-9c02-6ca31ed4ad3a',
 'source': '',
 'tags': None,
 'type': 'model',
 'versions': [{'created_by': 'mosaic.ai',
   'created_on': '2021-02-22T12:19:39+00:00',
   'datasource_name': '',
   'dependent_model': None,
   'deploy_info': None,
   'deployments': [],
   'description': None,
   'docker_image_url': 'registry.lti-aiq.in:443/mosaic-ai-logistics/mosaic-ai-serving:1.0.0-07122021',
   'gpu_docker_image_url': 'registry.lti-aiq.in:443/mosaic-ai-logistics/mosaic-ai-serving:gpu-1.0.0-07122021',
   'id': '071797a9-69e6-4dfe-900f-0d2539307824',
   'init_script': '"pip install --user absl-py==0.11.0\\n pip install -

In [ ]:

en = datetime.utcnow()
print(en,"\n")
diff = en - st
print(str(diff))

